In [8]:
import duckdb
import pyarrow as pa
import polars as pl
CHUNK_SIZE = 1000000

In [9]:

conn = duckdb.connect("timeball.db", read_only=True)
rel = conn.sql("SELECT * FROM main_models.stg_events")
batch_reader = rel.record_batch(CHUNK_SIZE)
schema = batch_reader.schema

def gen():
    for batch in batch_reader:
        df = pl.from_arrow(batch)
        # Do some stuff in polars, then go back to arrow
        arrow_table = df.to_arrow()
        temp_batcher = arrow_table.to_batches(CHUNK_SIZE)
        for new_batch in temp_batcher:
            yield new_batch

In [ ]:
final_batcher = pa.RecordBatchReader.from_batches(schema, gen())


In [10]:
import duckdb
import pyarrow as pa
from time import sleep
i = 0
conn = duckdb.connect("test.db")
def gen(batch_reader: pa.RecordBatchReader):
    global i
    for batch in batch_reader:
        i += batch.num_rows
        yield batch

def dbt_sim():
    print("Starting")
    cur = conn.cursor()
    cur.execute("CREATE OR REPLACE TABLE t AS (SELECT * AS var_col FROM UNNEST(generate_series(1, 10000000)))")
    o: duckdb.DuckDBPyRelation = cur.sql("SELECT * FROM t")
    print("created table")
    batch_reader: pa.RecordBatchReader = o.record_batch(10)
    print("created batch reader")
    new_reader = pa.RecordBatchReader.from_batches(batch_reader.schema, gen(batch_reader))
    print("test lazy exec")
    return new_reader

df = dbt_sim()
print("created df obj")
conn.execute("CREATE OR REPLACE TABLE t2 AS (SELECT * FROM df)")
conn.sql("SELECT COUNT(*) FROM t2")

Starting
created table
created batch reader
test lazy exec
created df obj


OutOfMemoryException: Out of Memory Error: could not allocate block of size 16KB (1.0GB/1.0GB used)